<a href="https://colab.research.google.com/github/soerenetler/qg_with_attention/blob/master/evaluation_set_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.system("git clone https://github.com/soerenetler/qg_with_attention.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import ast
dataset = "quac"

# Human Data

In [3]:
filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
human_df = pd.read_csv(filepath)[["question_text", "answer_sentence", "answer"]]

In [4]:
human_df = human_df.sample(100)
human_df.to_csv("h_human_evaluation.csv")

# E2E Data

In [6]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_quac-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_2-dropout_0.3-pretrained_True-bidirectional_True-answer_units_0-/test.txt"

e2e_df = pd.read_csv(filepath)[["answer_sentence", "answer"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))


e2e_df["question_text"] = pred_questions
e2e_df = e2e_df.sample(100)

In [7]:
e2e_df.head()

,answer_sentence,answer,question_text
25390,In 2003 he approached Amadou & Mariam and late...,In 2003 he approached Amadou & Mariam and late...,what was the name of deyoung 's solo album ?
25469,This decision was reached after a lagging fund...,lagging fundraising effort and polls that show...,did she win ?
18269,"The second single released from the album, ""Se...",Self Esteem,was the album successful ?
18172,"In a November 1, 2005, column for National Rev...","In a November 1, 2005, column for National Rev...",what else did he do ?
10376,"In real life, Ziegfeld did not die in his room...","In real life, Ziegfeld did not die in his room...",where did he live ?


In [8]:
e2e_df.to_csv("e2e_human_evaluation.csv")

# Answer Separated Data

In [9]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_quac-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/test.txt"


ans_df = pd.read_csv(filepath)[["answer_sentence", "answer"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))


ans_df["question_text"] = pred_questions

In [10]:
ans_df = ans_df.sample(100)

In [11]:
ans_df.to_csv("ans_human_evaluation.csv")

# Pipeline Data

In [12]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/trf_test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_quac-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/trf_test.txt"


pipe_df = pd.read_csv(data_filepath)[["answer_sentence_token", "answer"]]
pipe_df["answer_sentence_token"]  = [ast.literal_eval(t) for t in pipe_df["answer_sentence_token"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))

pipe_df["answer_sentence"] = [" ".join(t) for t in pipe_df["answer_sentence_token"]]
pipe_df.drop("answer_sentence_token", axis=1, inplace=True)
pipe_df["question_text"] = pred_questions
pipe_df = pipe_df.sample(100)

In [13]:
pipe_df.head()

,answer,answer_sentence,question_text
1999,"He had developed a gastric ulcer , and his hea...","He had developed a gastric ulcer , and his hea...",what else did he do ?
1949,In 1981 DC began publishing All - Star Squadro...,In 1981 DC began publishing All - Star Squadro...,what did he do next
446,He wrote three episodes of Murder Most Horrid ...,He wrote three episodes of Murder Most Horrid ...,what did he do next ?
1664,"He made his film debut in 1979 , playing Diane...","He made his film debut in 1979 , playing Diane...",what did he do after that ?
312,Mallory participated in the initial 1921 Briti...,Mallory participated in the initial 1921 Briti...,what did he do after that ?


In [14]:
pipe_df.to_csv("pipe_human_evaluation.csv")